In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [5]:
# Diccionario para estandarizar meses a números
meses_map = {
    'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4, 'MAYO': 5, 'JUNIO': 6,
    'JULIO': 7, 'AGOSTO': 8, 'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12
}

def limpiar_moneda(valor):
    """Convierte '100,000.00' o 80000 a float puro"""
    if pd.isna(valor): return 0.0
    # Convertir a string, quitar 'RD$', comas y espacios
    s = str(valor).replace('RD$', '').replace(',', '').strip()
    try:
        return float(s)
    except:
        return 0.0

def limpiar_texto(texto):
    """Quita espacios extra y pone todo en mayúsculas"""
    if pd.isna(texto): return "DESCONOCIDO"
    return str(texto).strip().upper()

print("Funciones configuradas.")

Funciones configuradas.


In [7]:
# Cargar IPC
df_ipc = pd.read_csv('./raw_data/ipc_base_1984-2025.csv', sep=';')

# Limpieza IPC
# Filtramos solo desde 2018 en adelante para coincidir con la nómina
df_ipc = df_ipc[df_ipc['PERIODO'] >= 2018].copy()

# Estandarizamos mes para poder cruzar
df_ipc['MES_LIMPIO'] = df_ipc['MES'].apply(limpiar_texto).map(meses_map)

# Seleccionamos solo lo necesario: Año, Mes numérico y el Índice
df_ipc_clean = df_ipc[['PERIODO', 'MES_LIMPIO', 'INDICE']].rename(columns={
    'PERIODO': 'ANIO',
    'MES_LIMPIO': 'MES_NUM',
    'INDICE': 'IPC'
})

print(f"IPC procesado: {len(df_ipc_clean)} registros (2018-2025)")
# Vista previa rápida
display(df_ipc_clean.head(10))

IPC procesado: 94 registros (2018-2025)


,ANIO,MES_NUM,IPC
408,2018,1,94.58
409,2018,2,94.48
410,2018,3,94.83
411,2018,4,95.21
412,2018,5,95.46
413,2018,6,95.66
414,2018,7,95.64
415,2018,8,95.67
416,2018,9,95.74
417,2018,10,95.96


In [27]:
# Cargar Nómina
df_nomina = pd.read_csv(
    './raw_data/nomina-empleados-fijos-y-contratados-CSV-2018-2025.csv',
    sep=';',
    encoding='latin-1',
    low_memory=False
)

# Normalizar nombres de columnas
df_nomina.columns = (
    df_nomina.columns
    .str.strip()
    .str.upper()
    .str.normalize("NFKD")
    .str.encode("ascii", errors="ignore")
    .str.decode("utf-8")
)

# Renombrar AO → ANIO (si aparece así)
df_nomina.columns = [c.replace('AO', 'ANIO') for c in df_nomina.columns]

# LIMPIAR SUELDO
df_nomina['SUELDO_LIMPIO'] = df_nomina['SUELDO'].apply(limpiar_moneda)

# LIMPIAR TEXTOS
cols_texto = ['NOMBRE', 'FUNCION', 'DEPARTAMENTO', 'ESTATUS']
for col in cols_texto:
    if col in df_nomina.columns:
        df_nomina[col] = df_nomina[col].astype(str).apply(limpiar_texto)

# LIMPIAR Y EXTRAER AÑO (solución al error)
df_nomina['ANIO'] = (
    df_nomina['ANIO']
    .astype(str)
    .str.extract(r'(\d{4})') # Extrae el primer año válido
)

df_nomina['ANIO'] = pd.to_numeric(df_nomina['ANIO'], errors='coerce')

#Corregir Ñ's de los nombres
df_nomina['NOMBRE'] = df_nomina['NOMBRE'].str.replace('Ï¿½', 'Ñ', regex=False)

# Estandarizar Mes a Número
df_nomina['MES_NUM'] = (
    df_nomina['MES']
    .astype(str)
    .apply(limpiar_texto)
    .map(meses_map)
)

# Eliminar filas inválidas (mes o año no legibles)
df_nomina.dropna(subset=['ANIO', 'MES_NUM'], inplace=True)

df_nomina['ANIO'] = df_nomina['ANIO'].astype(int)
df_nomina['MES_NUM'] = df_nomina['MES_NUM'].astype(int)

# RESULTADO
print(f"Nómina procesada: {len(df_nomina)} empleados.")
display(df_nomina.head(10))


Nómina procesada: 152728 empleados.


,NOMBRE,FUNCION,DEPARTAMENTO,SUELDO,ESTATUS,MES,ANIO,SUELDO_LIMPIO,MES_NUM
0,NOEL LUPERON RAMIREZ,ASESOR ACADEMICO,CENTRO DE CAPACITACION CGR,80000,CONTRATADO,ENERO,2018,80000.0,1
1,PAOLA MARITZA POLANCO RODRIGUEZ,COORDINADOR ACADEMICO,CENTRO DE CAPACITACION CGR,40000,FIJOS,ENERO,2018,40000.0,1
2,ANA IRIS MARTINEZ NUÑEZ,DIGITADOR(A),CONSULTORIA JURIDICA,20000,CONTRATADO,ENERO,2018,20000.0,1
3,ANGEL FRANCISCO ROMAN CORCINO,AUXILIAR ADMINISTRATIVO(A),CONSULTORIA JURIDICA,25000,FIJOS,ENERO,2018,25000.0,1
4,ANNETTE ALTAGRACIA PEÑA ACOSTA,ENCARGADO DIVISION,CONSULTORIA JURIDICA,60000,FIJOS,ENERO,2018,60000.0,1
5,CAROLINA ALCANTARA MERCEDES,ABOGADO (A) II,CONSULTORIA JURIDICA,50000,FIJOS,ENERO,2018,50000.0,1
6,DEMETRIO DECENA SANCHEZ,ABOGADO (A) II,CONSULTORIA JURIDICA,50000,FIJOS,ENERO,2018,50000.0,1
7,DOMINGO LUISOR ARIAS,AUDITOR LEGAL,CONSULTORIA JURIDICA,45000,FIJOS,ENERO,2018,45000.0,1
8,ELADIA DE JESUS MOREL,ABOGADO (A) II,CONSULTORIA JURIDICA,50000,FIJOS,ENERO,2018,50000.0,1
9,JIMMY DANIEL BONILLA HIDALGO,AUDITOR LEGAL,CONSULTORIA JURIDICA,45000,CONTRATADO,ENERO,2018,45000.0,1


In [28]:
# Unir Nómina con IPC usando Año y Mes
df_final = pd.merge(df_nomina, df_ipc_clean, on=['ANIO', 'MES_NUM'], how='left')

# ENRIQUECIMIENTO: Calcular Salario Real
# Fórmula: (Salario Nominal / IPC Actual) * IPC Base (Usaremos 100 como base conceptual o el IPC del primer registro)
# Para simplificar, ajustaremos a valor de 2018: Salario / IPC * IPC_2018
ipc_base = df_ipc_clean['IPC'].min() # Tomamos el menor IPC del periodo como base
df_final['SUELDO_REAL'] = (df_final['SUELDO_LIMPIO'] / df_final['IPC']) * ipc_base

# CARGA A SQLITE (Data Warehouse)
conn = sqlite3.connect('DW_Nomina_Publica.db')

# Crear Tabla de Hechos (Fact Table)
# Guardamos las métricas numéricas y claves
fact_table = df_final[['ANIO', 'MES_NUM', 'SUELDO_LIMPIO', 'SUELDO_REAL', 'IPC', 'FUNCION', 'DEPARTAMENTO', 'ESTATUS', 'NOMBRE']]
fact_table.to_sql('fact_nomina', conn, if_exists='replace', index=False)

print("Data Warehouse creado: 'DW_Nomina_Publica.db'")
conn.close()

Data Warehouse creado: 'DW_Nomina_Publica.db'
